AE part

In [8]:
datas = [

'''Experiment settings: FAAE. Average Report. Dataset av45
auc_roc: mean
0.6466697362890201
accuracy: mean
0.6408580652452546
sensitivity: mean
0.7494525010508618
specificity: mean
0.5359792924935289
significance_ratio: mean
1.6536209933476853
hz_para_list: [20, 20]'''

,

'''Experiment settings: FAAE. Average Report. Dataset fdg
auc_roc: mean
0.7360667352492202
accuracy: mean
0.6269722153421837
sensitivity: mean
0.6229287515762926
specificity: mean
0.7764567155593903
significance_ratio: mean
3.4736729865256657
hz_para_list: [20, 20]'''
,

'''
Experiment settings: FAAE. Average Report. Dataset vbm
auc_roc: mean
0.6072756963341518
accuracy: mean
0.5301690170295094
sensitivity: mean
0.6169283312316098
specificity: mean
0.5843025596778834
significance_ratio: mean
inf
hz_para_list: [20, 20]'''
,

'''
Experiment settings: FAAE. Average Report. Dataset snp
auc_roc: mean
0.5501122762770735
accuracy: mean
0.5329947383852373
sensitivity: mean
0.5802269861286254
specificity: mean
0.5601322979580098
significance_ratio: mean
1.3994988736972849
hz_para_list: [20, 20]'''

]

\begin{table}[h]
\centering
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & AUC-ROC & Accuracy & Sensitivity & Specificity \\
\hline
av45 & 64.67 & 64.09 & 74.95 & 53.6 \\
fdg & 73.61 & 62.7 & 62.29 & 77.65 \\
vbm & 60.73 & 53.02 & 61.69 & 58.43 \\
snp & 55.01 & 53.3 & 58.02 & 56.01 \\
\hline
\end{tabular}
\caption{AE Average Report}
\end{table}


In [49]:
# read data from result_baseline/result_optimal.txt

with open('result_baseline/result_optimal.txt', 'r') as f:
    file = f.read()
    datas = file.split('\n\n\n\n')


In [51]:
data['compare_name']

'MCI vs AD'

In [60]:
import re
from collections import defaultdict
from tabulate import tabulate

# Structuring the data
data_struct = defaultdict(lambda: defaultdict(dict))

for data in datas:
    # Extract the required information using regular expressions
    compare_name = re.search(r'Experiment settings: AE. (.+). Dataset', data).group(1)
    dataset_name = re.search(r'Dataset (.+)', data).group(1)

    # Extracting the mean and standard deviation for each metric
    AUC_ROC_mean, AUC_ROC_std = map(float, re.search(r'ROC-AUC: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    accuracy_mean, accuracy_std = map(float, re.search(r'Accuracy: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    sensitivity_mean, sensitivity_std = map(float, re.search(r'Sensitivity: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    specificity_mean, specificity_std = map(float, re.search(r'Specificity: \$ (\d+\.\d+) \\pm (\d+\.\d+) \$', data).groups())
    # Significance ratio extraction is commented out as it seems not required currently.
    hz_para_list = list(map(int, re.search(r'hz_para_list: \[(.+)\]', data).group(1).split(', ')))

    # Organizing the data by compare_name and dataset
    data_struct[compare_name][dataset_name] = {
        'ROC-AUC': (AUC_ROC_mean, AUC_ROC_std),
        'Accuracy': (accuracy_mean, accuracy_std),
        'Sensitivity': (sensitivity_mean, sensitivity_std),
        'Specificity': (specificity_mean, specificity_std),
        # 'Significance Ratio': (significance_ratio_mean, significance_ratio_std) # Uncomment if needed
        'hz_para_list': hz_para_list,
        'compare_name': compare_name
    }

# change _ in compare_name to space
for compare_name, datasets in data_struct.items():
    for dataset_name, data in datasets.items():
        data['compare_name'] = data['compare_name'].replace('_', ' ')
    


# Printing the data in a latex format, only print ROC-AUC, Accuracy, Sensitivity, Specificity
for compare_name, datasets in data_struct.items():
    print(f'\\begin{{table}}[H]')
    print(f'\\centering')
    compare_name = compare_name.replace('_', '-')
    print(f'\\caption{{Performance of {compare_name} on different datasets}}')
    print(f'\\label{{tab:my-table}}')
    print(f'\\begin{{tabular}}{{' + '|'.join(['c' for _ in range(len(datasets) + 1)]) + '}')
    print(f'\\hline')
    print(f'Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
    print(f'\\hline')
    for dataset_name, data in datasets.items():
        print(f'{dataset_name} & {data["ROC-AUC"][0]:.2f} & {data["Accuracy"][0]:.2f} & {data["Sensitivity"][0]:.2f} & {data["Specificity"][0]:.2f} \\\\')
    print(f'\\hline')
    print(f'\\end{{tabular}}')
    print(f'\\end{{table}}')
    print(f'\\vspace{{1cm}}')
    print(f'\\noindent')
    




\begin{table}[H]
\centering
\caption{Performance of HC-vs-AD on different datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & AUC-ROC & Accuracy & Sensitivity & Specificity \\
\hline
av45 & 72.26 & 77.04 & 84.52 & 56.58 \\
fdg & 85.74 & 77.18 & 75.00 & 83.16 \\
vbm & 66.32 & 64.86 & 66.35 & 60.79 \\
snp & 57.51 & 61.13 & 62.88 & 56.32 \\
\hline
\end{tabular}
\end{table}
\vspace{1cm}
\noindent
\begin{table}[H]
\centering
\caption{Performance of HC-vs-MCI on different datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & AUC-ROC & Accuracy & Sensitivity & Specificity \\
\hline
av45 & 55.34 & 58.08 & 58.39 & 55.53 \\
fdg & 57.99 & 41.69 & 36.59 & 82.63 \\
vbm & 53.26 & 42.94 & 39.21 & 72.89 \\
snp & 52.43 & 47.84 & 45.61 & 65.79 \\
\hline
\end{tabular}
\end{table}
\vspace{1cm}
\noindent
\begin{table}[H]
\centering
\caption{Performance of MCI-vs-AD on different datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c}
\hline
Dataset & AUC-ROC & A

FAAE part

In [268]:
import re

In [313]:
file = open('result_focal/result_focal_main_2.txt', 'r')
with open('result_focal/result_focal_main_2.txt', 'r') as f:
    file = f.read()
    # case sensitive
    datas = file.split('\n\n\n\ne')
for i in range(len(datas)):
    if datas[i].startswith('xperiment'):
        datas[i] = 'e' + datas[i]

In [314]:
import re

alpha_focal = []
gamma_focal = []
compare_names = ['MCI_vs_AD', 'HC_vs_MCI', 'HC_vs_AD']
results = {}

for data in datas:
    # Extract alpha_focal and gamma_focal
    alpha_match = re.search(r'args\.alpha_focal: (.+)', data)
    gamma_match = re.search(r'args\.gamma_focal: (.+)', data)
    if alpha_match and gamma_match:
        alpha_focal.append(float(alpha_match.group(1)))
        gamma_focal.append(float(gamma_match.group(1)))

    # Extract compare_name results
    for compare_name in compare_names:
        compare_match = re.search(r'Experiment settings: FAAE\. ' + compare_name + r'\. Dataset av45\nauc_roc: mean and std\n(.+) (.+)\naccuracy: mean and std\n(.+) (.+)\nsensitivity: mean and std\n(.+) (.+)\nspecificity: mean and std\n(.+) (.+)\nsignificance_ratio: mean and std\n(.+) (.+)\nhz_para_list: \[(.+)\]', data)
        if compare_match:
            auc_roc_mean = float(compare_match.group(1))
            auc_roc_std = float(compare_match.group(2))
            accuracy_mean = float(compare_match.group(3))
            accuracy_std = float(compare_match.group(4))
            sensitivity_mean = float(compare_match.group(5))
            sensitivity_std = float(compare_match.group(6))
            specificity_mean = float(compare_match.group(7))
            specificity_std = float(compare_match.group(8))
            significance_ratio_mean = float(compare_match.group(9))
            significance_ratio_std = float(compare_match.group(10))
            hz_para_list = list(map(int, compare_match.group(11).split(', ')))

            if compare_name not in results:
                results[compare_name] = []

            results[compare_name].append({
                'auc_roc_mean': auc_roc_mean,
                'auc_roc_std': auc_roc_std,
                'accuracy_mean': accuracy_mean,
                'accuracy_std': accuracy_std,
                'sensitivity_mean': sensitivity_mean,
                'sensitivity_std': sensitivity_std,
                'specificity_mean': specificity_mean,
                'specificity_std': specificity_std,
                'significance_ratio_mean': significance_ratio_mean,
                'significance_ratio_std': significance_ratio_std,
                'hz_para_list': hz_para_list
            })


# help me extract information. for data in datas, the data format is like above, attract information. each data have one alpha_focal and gamma_focal. remeber them. and each data have 3  compare_name MCI_vs_AD HC_vs_MCI HC_vs_AD. under each compare_name,\ there are auc_roc, accuracy, sensitivity, specificity, significance_ratio, hz_para_list, GET THEM RESPECTIVELY.





alpha_focal = []
gamma_focal = []
compare_names = ['MCI_vs_AD', 'HC_vs_MCI', 'HC_vs_AD']
results = {}

for data in datas:
    # Extract alpha_focal and gamma_focal
    alpha_match = re.search(r'args\.alpha_focal: (.+)', data)
    gamma_match = re.search(r'args\.gamma_focal: (.+)', data)
    if alpha_match and gamma_match:
        alpha_focal.append(float(alpha_match.group(1)))
        gamma_focal.append(float(gamma_match.group(1)))

    # Extract compare_name results
    for compare_name in compare_names:
        compare_match = re.search(r'Experiment settings: FAAE\. ' + compare_name + r'\. Dataset av45\nauc_roc: mean and std\n(.+) (.+)\naccuracy: mean and std\n(.+) (.+)\nsensitivity: mean and std\n(.+) (.+)\nspecificity: mean and std\n(.+) (.+)\nsignificance_ratio: mean and std\n(.+) (.+)\nhz_para_list: \[(.+)\]', data)
        if compare_match:
            auc_roc_mean = float(compare_match.group(1))
            auc_roc_std = float(compare_match.group(2))
            accuracy_mean = float(compare_match.group(3))
            accuracy_std = float(compare_match.group(4))
            sensitivity_mean = float(compare_match.group(5))
            sensitivity_std = float(compare_match.group(6))
            specificity_mean = float(compare_match.group(7))
            specificity_std = float(compare_match.group(8))
            significance_ratio_mean = float(compare_match.group(9))
            significance_ratio_std = float(compare_match.group(10))
            hz_para_list = list(map(int, compare_match.group(11).split(', ')))

            if compare_name not in results:
                results[compare_name] = []

            results[compare_name].append({
                'dataset_name': 'av45', # 'av45' is hardcoded here, you can extract it from the data using regex
                'auc_roc_mean': auc_roc_mean,
                'auc_roc_std': auc_roc_std,
                'accuracy_mean': accuracy_mean,
                'accuracy_std': accuracy_std,
                'sensitivity_mean': sensitivity_mean,
                'sensitivity_std': sensitivity_std,
                'specificity_mean': specificity_mean,
                'specificity_std': specificity_std,
                'significance_ratio_mean': significance_ratio_mean,
                'significance_ratio_std': significance_ratio_std,
                'hz_para_list': hz_para_list,
                'compare_name': compare_name,
                'alpha_focal': float(alpha_match.group(1)),
                'gamma_focal': float(gamma_match.group(1))
            })
        else:
            # compare_match = re.search(r'Experiment settings: FAAE\. ' + compare_name + r'\. Dataset (.+) \nauc_roc: mean\n(.+)\naccuracy: mean\n(.+)\nsensitivity: mean\n(.+)\nspecificity: mean\n(.+)\nsignificance_ratio: mean\n(.+)\nhz_para_list: \[(.+)\]', data)
            compare_match = re.search(r'Experiment settings: FAAE\. ' + compare_name + r'\. Dataset (.+)\nauc_roc: mean and std\n(.+) (.+)\naccuracy: mean and std\n(.+) (.+)\nsensitivity: mean and std\n(.+) (.+)\nspecificity: mean and std\n(.+) (.+)\nsignificance_ratio: mean and std\n(.+) (.+)\nhz_para_list: \[(.+)\]', data)
            if compare_match:
                dataset_name = compare_match.group(1)
                auc_roc_mean = float(compare_match.group(2))
                auc_roc_std = float(compare_match.group(3))
                accuracy_mean = float(compare_match.group(4))
                accuracy_std = float(compare_match.group(5))
                sensitivity_mean = float(compare_match.group(6))
                sensitivity_std = float(compare_match.group(7))
                specificity_mean = float(compare_match.group(8))
                specificity_std = float(compare_match.group(9))
                significance_ratio_mean = float(compare_match.group(10))
                significance_ratio_std = float(compare_match.group(11))
                hz_para_list = list(map(int, compare_match.group(12).split(', ')))

                if compare_name not in results:
                    results[compare_name] = []

                results[compare_name].append({
                    'dataset_name': dataset_name,
                    'auc_roc_mean': auc_roc_mean,
                    'auc_roc_std': auc_roc_std,
                    'accuracy_mean': accuracy_mean,
                    'accuracy_std': accuracy_std,
                    'sensitivity_mean': sensitivity_mean,
                    'sensitivity_std': sensitivity_std,
                    'specificity_mean': specificity_mean,
                    'specificity_std': specificity_std,
                    'significance_ratio_mean': significance_ratio_mean,
                    'significance_ratio_std': significance_ratio_std,
                    'hz_para_list': hz_para_list,
                    'compare_name': compare_name,
                    'alpha_focal': float(alpha_match.group(1)),
                    'gamma_focal': float(gamma_match.group(1))
                    
                })

            else:
                print(f'No match found for compare_name: {compare_name}')




No match found for compare_name: MCI_vs_AD


In [315]:
new_results = {}
for com in results:
    new_results[com] = [item for item in results[com] if item['hz_para_list'] == [20, 20]]
results = new_results



In [316]:
# if gamma is 30 or alpha is 1, remove the data
new_results = {}
for com in results:
    new_results[com] = [item for item in results[com] if (item['gamma_focal'] != 30 and item['alpha_focal'] != 1 and item['alpha_focal'] != 0.7)]
results = new_results


In [317]:
# multiply all number values by 100
for compare_name, compare_results in results.items():
    for result in compare_results:
        result['auc_roc_mean'] *= 100
        result['auc_roc_std'] *= 100
        result['accuracy_mean'] *= 100
        result['accuracy_std'] *= 100
        result['sensitivity_mean'] *= 100
        result['sensitivity_std'] *= 100
        result['specificity_mean'] *= 100
        result['specificity_std'] *= 100
        result['significance_ratio_mean'] *= 100
        result['significance_ratio_std'] *= 100
        result['alpha_focal'] = float(result['alpha_focal'])
        result['gamma_focal'] = float(result['gamma_focal'])

In [321]:
# for each compare_name, print a latex table

datasets = ['av45', 'fdg', 'vbm', 'snp']

for compare_name, compare_results in results.items():    
    for dataset_name in datasets:
        print(f'\\begin{{table}}[H]')
        print(f'\\centering')
        compare_name = compare_name.replace('_', '-')
        print(f'\\caption{{Performance of {compare_name} on {dataset_name} datasets}}')
        print(f'\\label{{tab:my-table}}')
        # print(f'\\begin{{tabular}}{{' + '|'.join(['c' for _ in range(len(compare_results) + 1)]) + '}')
        # print('') \begin{tabular}{c|c|c|c|c|c|c}
        print(f'\\begin{{tabular}}{{c|c|c|c|c|c|c}}')
        print(f'\\hline')
        # add alpha and gamma
        # print(f'Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
        print(f'alpha & gamma & Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
        print(f'\\hline')
        for result in compare_results:
            if result['dataset_name'] != dataset_name:
                continue
            # print(f'{result["dataset_name"]} & ${result["auc_roc_mean"]:.2f} \\pm {result["auc_roc_std"]:.2f}$ & ${result["accuracy_mean"]:.2f} \\pm {result["accuracy_std"]:.2f}$ & ${result["sensitivity_mean"]:.2f} \\pm {result["sensitivity_std"]:.2f}$ & ${result["specificity_mean"]:.2f} \\pm {result["specificity_std"]:.2f}$ \\\\')
            print(f'{result["alpha_focal"]} & {result["gamma_focal"]} & {result["dataset_name"]} & ${result["auc_roc_mean"]:.2f} \\pm {result["auc_roc_std"]:.2f}$ & ${result["accuracy_mean"]:.2f} \\pm {result["accuracy_std"]:.2f}$ & ${result["sensitivity_mean"]:.2f} \\pm {result["sensitivity_std"]:.2f}$ & ${result["specificity_mean"]:.2f} \\pm {result["specificity_std"]:.2f}$ \\\\')
        print(f'\\hline')
        print(f'\\end{{tabular}}')
        print(f'\\end{{table}}')
        print(f'\\vspace{{1cm}}')
        print(f'\\noindent')

\begin{table}[H]
\centering
\caption{Performance of MCI-vs-AD on av45 datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c|c|c}
\hline
alpha & gamma & Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
0.1 & 1.0 & av45 & $60.03 \pm 1.06$ & $48.41 \pm 3.95$ & $82.21 \pm 9.01$ & $36.89 \pm 8.23$ \\
0.1 & 5.0 & av45 & $59.61 \pm 1.76$ & $49.73 \pm 5.05$ & $79.23 \pm 10.23$ & $39.67 \pm 9.95$ \\
0.1 & 10.0 & av45 & $60.03 \pm 1.03$ & $51.27 \pm 4.71$ & $76.25 \pm 10.09$ & $42.75 \pm 9.61$ \\
0.1 & 20.0 & av45 & $60.10 \pm 0.75$ & $48.88 \pm 2.94$ & $81.54 \pm 5.23$ & $37.74 \pm 5.59$ \\
0.25 & 1.0 & av45 & $60.46 \pm 1.12$ & $51.91 \pm 3.27$ & $76.73 \pm 5.75$ & $43.44 \pm 6.20$ \\
0.25 & 5.0 & av45 & $60.56 \pm 1.78$ & $51.25 \pm 4.07$ & $77.88 \pm 7.80$ & $42.16 \pm 7.80$ \\
0.25 & 10.0 & av45 & $61.06 \pm 1.12$ & $51.69 \pm 5.52$ & $78.65 \pm 11.17$ & $42.49 \pm 11.04$ \\
0.25 & 20.0 & av45 & $60.10 \pm 1.04$ & $50.83 \pm 5.59$ & $77.60 \pm 9.87$ & $41.70 \pm 10.81$ 

In [322]:
# for each alpha and gamma and dataset pair, get a averge on compare_name

data1, data2, data3 = results['MCI_vs_AD'], results['HC_vs_MCI'], results['HC_vs_AD']



In [323]:
def ifdatassharesame(property):
    for i in range(len(data1)):
        # if data1[i][property] != data2[i][property] or data1[i][property] != data3[i][property]:
        if data1[i][property] == data2[i][property] and data1[i][property] == data3[i][property]:            
            continue
        else:
            print(data1[i][property], data2[i][property], data3[i][property])
    return True

In [324]:
if ifdatassharesame('dataset_name') and ifdatassharesame('alpha_focal') and ifdatassharesame('gamma_focal') and ifdatassharesame('hz_para_list'):
    print('all dataset names are the same')

all dataset names are the same


In [325]:
# create a new list, which get the average of the three compare_names

average_results = []

for i in range(len(data1)):
    average_results.append({
        'dataset_name': data1[i]['dataset_name'],
        'alpha_focal': data1[i]['alpha_focal'],
        'gamma_focal': data1[i]['gamma_focal'],
        'hz_para_list': data1[i]['hz_para_list'],
        'auc_roc_mean': (data1[i]['auc_roc_mean'] + data2[i]['auc_roc_mean'] + data3[i]['auc_roc_mean']) / 3,
        'auc_roc_std': (data1[i]['auc_roc_std'] + data2[i]['auc_roc_std'] + data3[i]['auc_roc_std']) / 3,
        'accuracy_mean': (data1[i]['accuracy_mean'] + data2[i]['accuracy_mean'] + data3[i]['accuracy_mean']) / 3,
        'accuracy_std': (data1[i]['accuracy_std'] + data2[i]['accuracy_std'] + data3[i]['accuracy_std']) / 3,
        'sensitivity_mean': (data1[i]['sensitivity_mean'] + data2[i]['sensitivity_mean'] + data3[i]['sensitivity_mean']) / 3,
        'sensitivity_std': (data1[i]['sensitivity_std'] + data2[i]['sensitivity_std'] + data3[i]['sensitivity_std']) / 3,
        'specificity_mean': (data1[i]['specificity_mean'] + data2[i]['specificity_mean'] + data3[i]['specificity_mean']) / 3,
        'specificity_std': (data1[i]['specificity_std'] + data2[i]['specificity_std'] + data3[i]['specificity_std']) / 3,
        'significance_ratio_mean': (data1[i]['significance_ratio_mean'] + data2[i]['significance_ratio_mean'] + data3[i]['significance_ratio_mean']) / 3,
        'significance_ratio_std': (data1[i]['significance_ratio_std'] + data2[i]['significance_ratio_std'] + data3[i]['significance_ratio_std']) / 3
    })


In [326]:
datasets = ['av45', 'fdg', 'vbm', 'snp']

for dataset_name in datasets:
    print(f'\\begin{{table}}[H]')
    print(f'\\centering')
    print(f'\\caption{{Performance of 3 average comparison on {dataset_name} datasets}}')
    print(f'\\label{{tab:my-table}}')
    # print(f'\\begin{{tabular}}{{' + '|'.join(['c' for _ in range(len(compare_results) + 1)]) + '}')
    # print('') \begin{tabular}{c|c|c|c|c|c|c}
    print(f'\\begin{{tabular}}{{c|c|c|c|c|c|c}}')
    print(f'\\hline')
    # add alpha and gamma
    # print(f'Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
    print(f'alpha & gamma & Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\\\')
    print(f'\\hline')
    for result in average_results:
        if result['dataset_name'] != dataset_name:
            continue
        print(f'{result["alpha_focal"]} & {result["gamma_focal"]} & {result["dataset_name"]} & ${result["auc_roc_mean"]:.2f} \\pm {result["auc_roc_std"]:.2f}$ & ${result["accuracy_mean"]:.2f} \\pm {result["accuracy_std"]:.2f}$ & ${result["sensitivity_mean"]:.2f} \\pm {result["sensitivity_std"]:.2f}$ & ${result["specificity_mean"]:.2f} \\pm {result["specificity_std"]:.2f}$ \\\\')
    print(f'\\hline')
    print(f'\\end{{tabular}}')
    print(f'\\end{{table}}')
    print(f'\\vspace{{1cm}}')
    print(f'\\noindent')

\begin{table}[H]
\centering
\caption{Performance of 3 average comparison on av45 datasets}
\label{tab:my-table}
\begin{tabular}{c|c|c|c|c|c|c}
\hline
alpha & gamma & Dataset & ROC-AUC & Accuracy & Sensitivity & Specificity \\
\hline
0.1 & 1.0 & av45 & $57.04 \pm 2.26$ & $52.58 \pm 7.76$ & $64.42 \pm 13.02$ & $52.56 \pm 11.80$ \\
0.1 & 5.0 & av45 & $56.59 \pm 2.47$ & $54.04 \pm 5.63$ & $64.35 \pm 9.96$ & $52.87 \pm 8.63$ \\
0.1 & 10.0 & av45 & $56.75 \pm 1.93$ & $53.99 \pm 4.67$ & $62.62 \pm 8.43$ & $54.43 \pm 8.55$ \\
0.1 & 20.0 & av45 & $56.74 \pm 1.69$ & $54.28 \pm 4.90$ & $65.49 \pm 7.67$ & $52.40 \pm 7.32$ \\
0.25 & 1.0 & av45 & $56.87 \pm 1.54$ & $54.41 \pm 6.16$ & $63.11 \pm 9.28$ & $54.31 \pm 8.23$ \\
0.25 & 5.0 & av45 & $57.28 \pm 2.41$ & $53.45 \pm 7.05$ & $61.90 \pm 10.84$ & $56.60 \pm 9.68$ \\
0.25 & 10.0 & av45 & $57.41 \pm 1.92$ & $56.14 \pm 5.52$ & $65.85 \pm 9.41$ & $53.73 \pm 8.96$ \\
0.25 & 20.0 & av45 & $56.82 \pm 1.98$ & $55.54 \pm 6.14$ & $65.22 \pm 9.78$ & $52.41 \